In [1]:
# load libraries
import lstm
import torch
import queue
import threading
import numpy as np 
import pandas as pd
import torch.nn as nn
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
from scipy import stats
import seaborn as sns
import os
from collections import namedtuple
import sys
import os
import numpy as np
from scipy.stats import spearmanr, kendalltau


print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
### create a dataset for the 6 sim scores from the combined weather dataset

dona_ana_df = pd.read_csv('../../../data/old_data/weather_data/cleaned_data/cleaned_dona_ana_df')
chaves_df = pd.read_csv('../../../data/old_data/weather_data/cleaned_data/cleaned_chaves_df')
otero_df = pd.read_csv('../../../data/old_data/weather_data/cleaned_data/cleaned_otero_df')
sierra_df = pd.read_csv('../../../data/old_data/weather_data/cleaned_data/cleaned_sierra_df')

dona_ana_df.head()


,dt,city_name,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description
0,1979-01-01 00:00:00+00:00,Doña Ana,32.393081,-106.815781,8.35,1.95,5.39,7.71,8.84,1013,64,5.35,250,12,801,Clouds,few clouds
1,1979-01-01 01:00:00+00:00,Doña Ana,32.393081,-106.815781,8.52,1.89,5.81,7.95,9.05,1014,63,4.83,248,3,800,Clear,sky is clear
2,1979-01-01 02:00:00+00:00,Doña Ana,32.393081,-106.815781,8.64,2.44,6.11,8.12,9.27,1014,65,4.48,249,1,800,Clear,sky is clear
3,1979-01-01 03:00:00+00:00,Doña Ana,32.393081,-106.815781,5.93,1.27,2.95,5.47,6.54,1016,72,4.10,253,2,800,Clear,sky is clear
4,1979-01-01 04:00:00+00:00,Doña Ana,32.393081,-106.815781,6.07,1.41,3.25,5.65,6.64,1016,72,3.86,252,1,800,Clear,sky is clear


In [3]:


df = pd.concat([dona_ana_df, chaves_df, otero_df, sierra_df] , ignore_index=True)
df = df.drop(['weather_main', 'weather_description'], axis=1)
df.head()

,dt,city_name,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id
0,1979-01-01 00:00:00+00:00,Doña Ana,32.393081,-106.815781,8.35,1.95,5.39,7.71,8.84,1013,64,5.35,250,12,801
1,1979-01-01 01:00:00+00:00,Doña Ana,32.393081,-106.815781,8.52,1.89,5.81,7.95,9.05,1014,63,4.83,248,3,800
2,1979-01-01 02:00:00+00:00,Doña Ana,32.393081,-106.815781,8.64,2.44,6.11,8.12,9.27,1014,65,4.48,249,1,800
3,1979-01-01 03:00:00+00:00,Doña Ana,32.393081,-106.815781,5.93,1.27,2.95,5.47,6.54,1016,72,4.10,253,2,800
4,1979-01-01 04:00:00+00:00,Doña Ana,32.393081,-106.815781,6.07,1.41,3.25,5.65,6.64,1016,72,3.86,252,1,800


In [4]:

df['date'] = pd.to_datetime(df['dt'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

df = df.drop(['date', 'dt'], axis=1)
df.head()

,city_name,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,year,month,day
0,Doña Ana,32.393081,-106.815781,8.35,1.95,5.39,7.71,8.84,1013,64,5.35,250,12,801,1979,1,1
1,Doña Ana,32.393081,-106.815781,8.52,1.89,5.81,7.95,9.05,1014,63,4.83,248,3,800,1979,1,1
2,Doña Ana,32.393081,-106.815781,8.64,2.44,6.11,8.12,9.27,1014,65,4.48,249,1,800,1979,1,1
3,Doña Ana,32.393081,-106.815781,5.93,1.27,2.95,5.47,6.54,1016,72,4.10,253,2,800,1979,1,1
4,Doña Ana,32.393081,-106.815781,6.07,1.41,3.25,5.65,6.64,1016,72,3.86,252,1,800,1979,1,1


In [5]:
weather_df = df
weather_df = weather_df[(weather_df['year'] >= 2016) & (weather_df['year'] <= 2022)]

In [6]:
#load the crop yield data and get an idea of what it looks like
yield_df = pd.read_csv('../../../data/pecan/yield_05_23_24.csv')
# we do not need hte state, ag distrinct, ag district code, coundy ansi or commodidy 
columns_to_drop = ['State', 'Ag District', 'Ag District Code', 'County ANSI', 'Commodity']
yield_df.drop(columns=columns_to_drop, inplace=True)
#yield values are strings, so i will convert them to floats
yield_df['Value'] = yield_df['Value'].str.replace(',', '').astype(float)
#convert the year to a datetime object
yield_df['Year'] = pd.to_datetime(yield_df['Year'], format='%Y')


In [7]:
combined_df = weather_df.copy()  # Create a copy of the DataFrame to avoid modifying the original
for index, row in yield_df.iterrows():
    year = row['Year'].year
    county = row['County']
    value = row['Value']
    mask = (combined_df['year'] == year) & (combined_df['city_name'] == county)
    combined_df.loc[mask, 'yield'] = value


In [8]:
print(combined_df.shape)
combined_df.head()

df = combined_df.copy()

x = df.drop(['yield', 'city_name'], axis=1)
y = df[['yield', 'city_name']]

(245472, 18)


In [9]:
print(x.shape)
x.head()

(245472, 16)


,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,year,month,day
324336,32.393081,-106.815781,8.99,1.42,7.93,7.60,9.46,1019,59,2.10,20,90,804,2016,1,1
324337,32.393081,-106.815781,8.25,2.07,7.63,7.44,8.68,1020,65,1.52,51,87,804,2016,1,1
324338,32.393081,-106.815781,7.37,1.44,6.04,6.10,7.80,1021,66,2.10,340,81,803,2016,1,1
324339,32.393081,-106.815781,5.58,1.32,3.13,4.30,6.02,1022,74,3.10,350,40,802,2016,1,1
324340,32.393081,-106.815781,5.43,0.79,5.43,4.75,5.90,1023,72,1.17,48,87,804,2016,1,1


In [10]:
print(y.shape)
y.head()

(245472, 2)


,yield,city_name
324336,1837.0,Doña Ana
324337,1837.0,Doña Ana
324338,1837.0,Doña Ana
324339,1837.0,Doña Ana
324340,1837.0,Doña Ana


In [11]:
scaler = MinMaxScaler()
scaled_x = scaler.fit_transform(x)
x_df = pd.DataFrame(scaled_x, columns=x.columns)

print(x_df.shape)
x_df.head()


(245472, 16)


,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,year,month,day
0,0.0,0.108569,0.450352,0.583306,0.500297,0.436465,0.457344,0.500000,0.585859,0.088645,0.055556,0.90,1.000000,0.0,0.0,0.0
1,0.0,0.108569,0.438780,0.594092,0.495840,0.433917,0.445156,0.517857,0.646465,0.064162,0.141667,0.87,1.000000,0.0,0.0,0.0
2,0.0,0.108569,0.425020,0.583638,0.472214,0.412580,0.431406,0.535714,0.656566,0.088645,0.944444,0.81,0.998344,0.0,0.0,0.0
3,0.0,0.108569,0.397029,0.581646,0.428975,0.383917,0.403594,0.553571,0.737374,0.130857,0.972222,0.40,0.996689,0.0,0.0,0.0
4,0.0,0.108569,0.394683,0.572851,0.463150,0.391083,0.401719,0.571429,0.717172,0.049388,0.133333,0.87,1.000000,0.0,0.0,0.0


In [12]:
x_df = x_df.reset_index(drop=True)
y = y.reset_index(drop=True)


In [13]:
df = pd.concat([x_df, y],axis=1)
print(df.shape)
df.head()

(245472, 18)


,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,year,month,day,yield,city_name
0,0.0,0.108569,0.450352,0.583306,0.500297,0.436465,0.457344,0.500000,0.585859,0.088645,0.055556,0.90,1.000000,0.0,0.0,0.0,1837.0,Doña Ana
1,0.0,0.108569,0.438780,0.594092,0.495840,0.433917,0.445156,0.517857,0.646465,0.064162,0.141667,0.87,1.000000,0.0,0.0,0.0,1837.0,Doña Ana
2,0.0,0.108569,0.425020,0.583638,0.472214,0.412580,0.431406,0.535714,0.656566,0.088645,0.944444,0.81,0.998344,0.0,0.0,0.0,1837.0,Doña Ana
3,0.0,0.108569,0.397029,0.581646,0.428975,0.383917,0.403594,0.553571,0.737374,0.130857,0.972222,0.40,0.996689,0.0,0.0,0.0,1837.0,Doña Ana
4,0.0,0.108569,0.394683,0.572851,0.463150,0.391083,0.401719,0.571429,0.717172,0.049388,0.133333,0.87,1.000000,0.0,0.0,0.0,1837.0,Doña Ana


In [14]:
dona_ana_df = df[df['city_name'] == 'Doña Ana']
chaves_df = df[df['city_name'] == 'Chaves County']
otero_df= df[df['city_name'] == 'Otero County']
sierra_df = df[df['city_name'] == 'Sierra County']


In [17]:
df.columns

Index(['lat', 'lon', 'temp', 'dew_point', 'feels_like', 'temp_min', 'temp_max',
       'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all',
       'weather_id', 'year', 'month', 'day', 'yield', 'city_name'],
      dtype='object')

In [15]:
print(dona_ana_df.shape)
print(chaves_df.shape)
print(otero_df.shape)
print(sierra_df.shape)

(61368, 18)
(61368, 18)
(61368, 18)
(61368, 18)


In [16]:
columns = ['lat', 'lon', 'temp', 'dew_point', 'feels_like', 'temp_min', 'temp_max',
       'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all',
       'weather_id', 'year', 'month', 'day', ]

df = pd.Dataframe()


def cosine(num1, num2):
    return cosine_similarity(num1, num2)
def euclid(num1, num2):
    return distance.euclidean(num1, num2)
def manhattan(num1, num2):
    return distance.cityblock(num1,num2)
def pearson(num1, num2):
    dist, _ = stats.pearsonr(num1,num2)
    return dist
def spearman(num1, num2):
    dist, _ = stats.spearmanr(num1, num2)
    return dist
def kendall_tau(num1, num2):
    return stats.kendalltau(num1, num2)

sim_functions = [cosine, euclid, manhattan, pearson, spearman, kendall_tau,]

for col in columns:
    for simdex, funct in enumerate(sim_functions):
        similarities = []
        for valdex in range(len(dona_ana_df[col])):
            similarities.append(funct(dona_ana_df, chaves_df))
        df[]
        



In [18]:

dataframes = [dona_ana_df, chaves_df, otero_df, sierra_df]
columns = ['lat', 'lon', 'temp', 'dew_point', 'feels_like', 'temp_min', 'temp_max',
           'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all',
           'weather_id', 'year', 'month', 'day']

def cosine(num1, num2):
    return cosine_similarity(num1.reshape(1, -1), num2.reshape(1, -1))[0][0]

def euclid(num1, num2):
    return distance.euclidean(num1, num2)

def manhattan(num1, num2):
    return distance.cityblock(num1, num2)

def pearson(num1, num2):
    dist, _ = stats.pearsonr(num1, num2)
    return dist

def spearman(num1, num2):
    dist, _ = stats.spearmanr(num1, num2)
    return dist

def kendall_tau(num1, num2):
    return stats.kendalltau(num1, num2)[0]

sim_functions = [cosine, euclid, manhattan, pearson, spearman, kendall_tau]

# Prepare an empty dataframe to store the results
result_df = pd.DataFrame()

# Calculate similarities
for i in range(len(dataframes)):
    for j in range(i + 1, len(dataframes)):
        for col in columns:
            for sim_func in sim_functions:
                sim_col_name = f"{col}_{sim_func.__name__}_{i}_{j}"
                similarities = []
                for k in range(len(dataframes[i])):
                    similarities.append(sim_func(dataframes[i][col].iloc[k], dataframes[j][col].iloc[k]))
                result_df[sim_col_name] = similarities

# Reshape the result_df to the desired shape
result_df = result_df.values.reshape(-1, 576)

# Convert back to DataFrame
result_df = pd.DataFrame(result_df)

# Display the resulting dataframe
df.head()

ValueError: Input vector should be 1-D.

In [19]:

dataframes = [dona_ana_df, chaves_df, otero_df, sierra_df]
columns = ['lat', 'lon', 'temp', 'dew_point', 'feels_like', 'temp_min', 'temp_max',
           'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all',
           'weather_id', 'year', 'month', 'day']

def cosine(num1, num2):
    return cosine_similarity(num1.reshape(1, -1), num2.reshape(1, -1))[0][0]

def euclid(num1, num2):
    return distance.euclidean(num1, num2)

def manhattan(num1, num2):
    return distance.cityblock(num1, num2)

def pearson(num1, num2):
    dist, _ = stats.pearsonr(num1, num2)
    return dist

def spearman(num1, num2):
    dist, _ = stats.spearmanr(num1, num2)
    return dist

def kendall_tau(num1, num2):
    return stats.kendalltau(num1, num2)[0]

sim_functions = [cosine, euclid, manhattan, pearson, spearman, kendall_tau]

# Prepare an empty dataframe to store the results
result_df = pd.DataFrame()

# Calculate similarities
for i in range(len(dataframes)):
    for j in range(i + 1, len(dataframes)):
        for col in columns:
            for sim_func in sim_functions:
                sim_col_name = f"{col}_{sim_func.__name__}_{i}_{j}"
                similarities = []
                for k in range(len(dataframes[i])):
                    similarities.append(sim_func(dataframes[i][col].values, dataframes[j][col].values))
                result_df[sim_col_name] = similarities

# Reshape the result_df to the desired shape
result_df = result_df.values.reshape(-1, 576)

# Convert back to DataFrame
result_df = pd.DataFrame(result_df)



/home/intellect/anaconda3/envs/interpolate/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/intellect/anaconda3/envs/interpolate/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/intellect/anaconda3/envs/interpolate/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/intellect/anaconda3/envs/interpolate/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
